1. 썸네일에 있는 이미지 전체
 - url 패턴 분석
2. 브랜드, 품번
3. 시즌, 성별
4. 좋아요 수
5. 리뷰
 - 스타일 후기, 상품 후기, 일반 후기 크롤링

In [329]:
import requests
from bs4 import BeautifulSoup
import json

In [314]:
url = "https://www.musinsa.com/category/003009"
res = requests.get(url)
page = res.content
soup = BeautifulSoup(page, 'html.parser')

prd_list = []

thum_list_list = []
brand_name_list = []
brand_code_list = []
season_list = []
gender_list = []
like_cnt_list = []
comment_style_list = []
comment_product_list = []
comment_general_list = []

for j in soup.find_all("li",{"class":"li_box"}):
  prd_list.append(j["data-no"])

for j in prd_list:
    
    url = f"https://store.musinsa.com/app/goods/{j}"
    response = requests.get(url)
    res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    page = res.content
    soup = BeautifulSoup(page, 'html.parser')
    lst1 = soup.find("div",{"id" : "detail_thumb" })
    lst1.find_all("img")
    thum_list = []
    for i in lst1.find_all("img"):
      thum_list.append("https:"+i["src"].replace("_60","_500"))
    brand_name, brand_code = soup.find("div", {"class":"explan_product product_info_section"}).find_all("p", {"class":"product_article_contents"})[0].text.split(" / ")
    soup.find("p", {"class":"product_article_contents"}).text
    if len(soup.find("span", {"class":"txt_gender"}).parent.text.strip().replace("\n",",")) <= 3:
      season = "NaN"
    else:
      season = soup.find("span", {"class":"txt_gender"}).parent.find("strong").text.replace(" ","").strip()
    gender = soup.find("span",{"class":"txt_gender"}).text.strip().replace("\n",",")

    API_URL = "https://like.musinsa.com/like/api/v1/liketypes/goods/counts"
    params ={
        "relationIds":[f"{j}"]
    }
    
    headers = {
        "content-type" : "application/json",
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
    }
    response = requests.post(
        API_URL,
        data = json.dumps(params),
        headers = headers
    )
    like_cnt = response.json()['data']['contents'][0]["count"]
    
    url = f"https://goods.musinsa.com/api/goods/v1/review/style/list?similarNo=0&sort=new&selectedSimilarNo=0&page=1&goodsNo={j}&_=1650269905193"
    comments = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    page = comments.content
    soup = BeautifulSoup(page, 'html.parser')
    a2 = soup.find_all('div', {'class':"review-contents__text"})
    comment_style=[]
    for i in a2:
      comment_style.append(i.text)
    
    url = f"https://goods.musinsa.com/api/goods/v1/review/photo/list?similarNo=0&sort=new&selectedSimilarNo=0&page=1&goodsNo={j}&_=1650269905195"
    comments = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    page = comments.content
    soup = BeautifulSoup(page, 'html.parser')
    a2 = soup.find_all('div', {'class':"review-contents__text"})
    comment_product=[]
    for i in a2:
      comment_product.append(i.text)
    
    url = f"https://goods.musinsa.com/api/goods/v1/review/goods/list?similarNo=0&sort=new&selectedSimilarNo=0&page=1&goodsNo={j}&_=1650269905197"
    comments = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    page = comments.content
    soup = BeautifulSoup(page, 'html.parser')
    a2 = soup.find_all('div', {'class':"review-contents__text"})
    comment_general=[]
    for i in a2:
      comment_general.append(i.text)
    
    thum_list_list.append(thum_list)
    brand_name_list.append(brand_name)
    brand_code_list.append(brand_code)
    season_list.append(season)
    gender_list.append(gender)
    like_cnt_list.append(like_cnt)
    comment_style_list.append(comment_style)
    comment_product_list.append(comment_product)
    comment_general_list.append(comment_general)

In [253]:
import pandas as pd

In [324]:
data_dict = {
    "항목번호"   : prd_list,
    "썸네일사진" : thum_list_list,
    "브랜드명"   : brand_name_list,
    "품목번호"   : brand_code_list,
    "시즌"       : season_list,
    "성별"       : gender_list,
    "좋아요"     : like_cnt_list,
    "스타일후기" : comment_style_list,
    "상품후기"   : comment_product_list,
    "일반후기"   : comment_general_list
}
df = pd.DataFrame(data_dict)

In [330]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   항목번호    90 non-null     object
 1   썸네일사진   90 non-null     object
 2   브랜드명    90 non-null     object
 3   품목번호    90 non-null     object
 4   시즌      90 non-null     object
 5   성별      90 non-null     object
 6   좋아요     90 non-null     int64 
 7   스타일후기   90 non-null     object
 8   상품후기    90 non-null     object
 9   일반후기    90 non-null     object
dtypes: int64(1), object(9)
memory usage: 7.2+ KB


In [326]:
df.tail()

,항목번호,썸네일사진,브랜드명,품목번호,시즌,성별,좋아요,스타일후기,상품후기,일반후기
85,2490668,[https://image.msscdn.net/images/goods_img/202...,FP142,2221979,2022S/S,남,36,[],[],[]
86,1882336,[https://image.msscdn.net/images/goods_img/202...,UMBRO,UN223CHP41_BLK,2022S/S,남,1457,[후기 보고 신축성 없는건 알았는데 진짜 없어요 신축성이ㅋ 그래도 사이즈는 맞아서 ...,[안에 메쉬로 되어있는점은 좋아요 시원할거 같은데 대신 신축성은 없어요 평상시에 편...,[후기 보고 신축성 없는건 알았는데 진짜 없어요 신축성이ㅋㅋ 운동할땐 오히려 그래서...
87,1920011,[https://image.msscdn.net/images/goods_img/202...,UNIFORM BRIDGE,UB-2437,2021S/S,남,6091,"[흰티에 찰떡 약간 두꺼워서 여름에 더울거 같아요, 사이즈도 적당하게 잘맞고색감도 ...","[아주 예쁜 청 반바지네요. 핏도 예쁘고 여름내내 잘 입을 것 같습니다!, 바지 모...","[아주 예쁜 청 반바지네요. 핏도 예쁘고 여름내내 잘 입을 것 같습니다!, 이뻐서 ..."
88,2483836,[https://image.msscdn.net/images/goods_img/202...,LAFUDGESTORE,fudge,2022S/S,남,103,[],[],[]
89,2486528,[https://image.msscdn.net/images/goods_img/202...,LECOQ,QN223THP55,2022S/S,남,8,[],[],[]
